# "2020 April One Day Holding Long Short Portfolio BackTest"
> "2020 April Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [185]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt
import datetime

In [186]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\20204dayreturnpred2019train60K.csv')

In [187]:
#hide
df_testp=df_testp.loc[df_testp.Date>=20200401].loc[df_testp.Date<20200501].sort_values(by='Date',ascending=True).copy()

**Long-Short Portfolio**

In [188]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']/100))[:l])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [189]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1
0,20200401,1.0,-0.436200,1.000000
1,20200402,1.0,-12.519562,0.995638
2,20200403,1.0,2.484180,0.870988
3,20200406,1.0,-4.938272,0.892625
4,20200407,1.0,10.122484,0.848545
5,20200409,1.0,0.000000,0.934439
6,20200410,1.0,-1.374604,0.934439
7,20200413,1.0,-18.100880,0.921594
8,20200414,-1.0,0.000000,0.754777
9,20200414,1.0,-2.412551,0.754777


In [190]:
#hide_input
import altair as alt
import datetime
df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
alt.Chart(df_agg).mark_trail().encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1'
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)

**Long portfolio**

In [191]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']/100))[:l])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [192]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1
0,20200401,1.0,-0.436200,1.000000
1,20200402,1.0,-12.519562,0.995638
2,20200403,1.0,2.484180,0.870988
3,20200406,1.0,-4.938272,0.892625
4,20200407,1.0,10.122484,0.848545
5,20200409,1.0,0.000000,0.934439
6,20200410,1.0,-1.374604,0.934439
7,20200413,1.0,-18.100880,0.921594
8,20200414,1.0,-2.412551,0.754777
9,20200415,1.0,-35.537519,0.736568


In [193]:
#hide_input
import altair as alt
import datetime
df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
alt.Chart(df_agg).mark_trail().encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1'
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)

**Short Portfolio**

In [194]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']/100))[:l])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [195]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1
0,20200414,-1.0,0.0,1.0


In [196]:
#hide_input
import altair as alt
import datetime
df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
alt.Chart(df_agg).mark_trail().encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1'
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)